In [1]:
from sklearn import metrics
from tensorflow import keras
from sklearn.metrics.pairwise import cosine_similarity
import gensim.downloader
from tqdm import tqdm
import numpy as np
import string
import csv 
import re

In [2]:
np.random.seed(0)

import datetime
t_delta = datetime.timedelta(hours=9)
JST = datetime.timezone(t_delta, 'JST')
now = datetime.datetime.now(JST)

In [3]:
# parameters
SAVED_MODEL = "../model/Proposed-Word2Vec-TextCNN_"+str(now.strftime('%Y%m%d%H%M%S'))
THRESHOLD = 0.05
MAXLEN_GET_PSEUDO = 3000
EPOCH = 5
BATCH_SIZE = 64
MAX_LEN = 128
FILTERS = 100
FILTER_SIZES = [3,4,5]

In [4]:
print(SAVED_MODEL)

../model/Proposed-Word2Vec-TextCNN_20221127203942


In [5]:
# 前処理
def preprocessing(text):
    # 括弧内文章の削除
    text = re.sub(r'\(.*\)',' ',text)
    text = re.sub(r'\[.*\]',' ',text)
    text = re.sub(r'\<.*\>',' ',text)
    text = re.sub(r'\{.*\}',' ',text)
    # 記号文字の削除
    text = text.translate(str.maketrans('','',string.punctuation))
    # スペースの調整
    text = re.sub(r'\s+',' ',text)
    return text

In [6]:
# 20 newsgroups datasets
from sklearn.datasets import fetch_20newsgroups
newsgroups = fetch_20newsgroups(subset="all")
newsgroups_datasets = list()

# # example ------------------------------------------------
# for texts in tqdm(newsgroups.data[:1000]):
#   texts = texts.split("\n\n")
#   texts = " ".join(texts[1:])
#   newsgroups_datasets.append(preprocessing(texts))
# # --------------------------------------------------------

for texts in tqdm(newsgroups.data):
  texts = texts.split("\n\n")
  texts = " ".join(texts[1:])
  newsgroups_datasets.append(preprocessing(texts))

100%|██████████| 18846/18846 [00:01<00:00, 18373.49it/s]


In [7]:
# yahoo topic datasets
with open('../data/topic/train_pu_half_v0.txt','r',encoding='utf-8') as f:
    texts_v0 = f.read()
with open('../data/topic/train_pu_half_v1.txt','r',encoding='utf-8') as f:
    texts_v1 = f.read()
texts = texts_v0 + texts_v1
topic_datasets = list()

# # example ----------------------------------------------
# for label_text in tqdm(texts.splitlines()[:1000]):
#   _, text = label_text.split("\t")
#   topic_datasets.append(preprocessing(text))
# # -------------------------------------------------------

for label_text in tqdm(texts.splitlines()):
  _, text = label_text.split("\t")
  topic_datasets.append(preprocessing(text))

100%|██████████| 1300000/1300000 [00:27<00:00, 46751.41it/s]


In [8]:
# reuters datasets
with open("../data/reuter/sourceall.txt", "r", encoding="utf-8") as f:
  reuter = f.read().split("\n")[:-1]

# # example -------------------
# import random
# reuter = random.sample(reuter, 1000)
# #----------------------------

reuters_datasets = list()
for label_text in tqdm(reuter):
  _, text = label_text.split("\t")
  reuters_datasets.append(preprocessing(text))

100%|██████████| 762027/762027 [00:31<00:00, 24171.99it/s]


In [9]:
# dbpedia datasets train
with open('../data/dbpedia_csv/train.csv','r',encoding='utf-8') as f:
    reader = [r for r in csv.reader(f)]
    
# # example -------------------
# import random
# reader = random.sample(reader, 1000)
# #----------------------------

dbpedia_train_datasets = list()
for _, auth, text in tqdm(reader):
    text = text.replace(auth,'')
    dbpedia_train_datasets.append(preprocessing(text))

100%|██████████| 560000/560000 [00:08<00:00, 63445.93it/s]


In [10]:
# dbpedia classes
with open("../data/dbpedia_csv/classes.txt", "r", encoding="utf-8") as f:
  classes = f.read().splitlines()

In [11]:
datasets_texts = newsgroups_datasets + topic_datasets + reuters_datasets + dbpedia_train_datasets

In [12]:
# load test data
# dbpedia datasets train
with open('../data/dbpedia_csv/test.csv','r',encoding='utf-8') as f:
    reader = [r for r in csv.reader(f)]
    
# # example -------------------
# import random
# reader = random.sample(reader, 10000)
# #----------------------------

test_texts = list()
test_labels = list()
for labels, auth, text in tqdm(reader):
    text = text.replace(auth,'')
    test_texts.append(preprocessing(text))
    test_labels.append(int(labels)-1)

100%|██████████| 70000/70000 [00:00<00:00, 70498.82it/s]


# Choice method

In [13]:
word2vec = gensim.downloader.load('word2vec-google-news-300')

def w2v_avg_vector(sentence):
  vector = np.zeros((300,), dtype="float32")
  count = 0
  for word in sentence.split():
    try:
      vector = np.add(vector, word2vec[word])
      count += 1
    except:
      pass
  if count > 0:
    vector = np.divide(vector, len(word))
  return vector

In [14]:
classes_vector = [w2v_avg_vector(cls) for cls in classes]

In [15]:
diff_datasets = {i:[] for i in range(len(classes))}
for texts in tqdm(datasets_texts):
  texts_vector = w2v_avg_vector(texts)
  similarity = cosine_similarity([texts_vector], classes_vector)[0]
  sim_argsorted = np.argsort(similarity)
  diff = similarity[sim_argsorted[-1]] - similarity[sim_argsorted[-2]]
  if diff > THRESHOLD:
    diff_datasets[sim_argsorted[-1]].append((similarity[sim_argsorted[-1]], texts))

pseudo_texts = list()
pseudo_labels = list()
for i in range(len(classes)):
  sorted_diff_data = sorted(diff_datasets[i], reverse=True)[:MAXLEN_GET_PSEUDO]
  pseudo_texts.extend([i[1] for i in sorted_diff_data])
  pseudo_labels.extend([i]*len(sorted_diff_data[:MAXLEN_GET_PSEUDO]))

100%|██████████| 2640873/2640873 [10:25<00:00, 4225.32it/s]


In [16]:
print("Number of all selected data")
for i in diff_datasets:
  print(classes[i][:3]+". : "+str(len(diff_datasets[i])))

Number of all selected data
Com. : 61181
Edu. : 34672
Art. : 6818
Ath. : 16123
Off. : 126858
Mea. : 144368
Bui. : 10120
Nat. : 38562
Vil. : 35431
Ani. : 4742
Pla. : 13933
Alb. : 42379
Fil. : 26335
Wri. : 23263


# Train

In [17]:
tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(pseudo_texts)
tokenizer.fit_on_texts(test_texts)
embedding_matrix = np.zeros((max(tokenizer.word_index.values())+1, word2vec.vector_size))
for word, i in tokenizer.word_index.items():
    if word in word2vec:
        embedding_matrix[i] = word2vec[word]

sequences = tokenizer.texts_to_sequences(pseudo_texts)
x_train = keras.preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_LEN, padding='post')
y_train = np.array(keras.utils.to_categorical(pseudo_labels))

sequences = tokenizer.texts_to_sequences(test_texts)
x_test = keras.preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_LEN, padding='post')
y_test = np.array(keras.utils.to_categorical(test_labels))

In [18]:
inputs = keras.layers.Input(shape=(MAX_LEN),dtype='float32')
embedding = keras.layers.Embedding(input_dim=len(embedding_matrix),output_dim=len(embedding_matrix[0]),weights=[embedding_matrix],trainable=False, mask_zero=True,name='embedding')(inputs)
conv1 = keras.layers.Conv1D(filters=FILTERS, kernel_size=FILTER_SIZES[0], kernel_initializer='normal', activation='relu')(embedding)
conv2 = keras.layers.Conv1D(filters=FILTERS, kernel_size=FILTER_SIZES[1], kernel_initializer='normal', activation='relu')(embedding)
conv3 = keras.layers.Conv1D(filters=FILTERS, kernel_size=FILTER_SIZES[2], kernel_initializer='normal', activation='relu')(embedding)
pool1 = keras.layers.MaxPooling1D(pool_size=int(conv1.shape[1]),strides=1)(conv1)
pool2 = keras.layers.MaxPooling1D(pool_size=int(conv2.shape[1]),strides=1)(conv2)
pool3 = keras.layers.MaxPooling1D(pool_size=int(conv3.shape[1]),strides=1)(conv3)
x = keras.layers.concatenate([pool1, pool2, pool3])
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(100,activation='relu')(x)
output = keras.layers.Dense(units=14, activation='softmax')(x)
model = keras.models.Model(inputs, output)
opt = keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=True)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [19]:
# モデルの学習
result = model.fit(x=x_train, y=y_train, epochs=EPOCH, batch_size=BATCH_SIZE, validation_data=(x_test,y_test))

Epoch 1/5
657/657 [==============================] - 45s 67ms/step - loss: 1.3669 - accuracy: 0.6215 - val_loss: 1.0081 - val_accuracy: 0.7081
Epoch 2/5
657/657 [==============================] - 47s 71ms/step - loss: 0.3961 - accuracy: 0.8945 - val_loss: 0.9329 - val_accuracy: 0.6785
Epoch 3/5
657/657 [==============================] - 46s 70ms/step - loss: 0.2493 - accuracy: 0.9305 - val_loss: 0.9854 - val_accuracy: 0.6659
Epoch 4/5
657/657 [==============================] - 47s 71ms/step - loss: 0.1918 - accuracy: 0.9448 - val_loss: 1.0161 - val_accuracy: 0.6717
Epoch 5/5
657/657 [==============================] - 47s 71ms/step - loss: 0.1584 - accuracy: 0.9545 - val_loss: 1.1098 - val_accuracy: 0.6532


In [20]:
model.save(SAVED_MODEL)

INFO:tensorflow:Assets written to: ../model/Proposed-Word2Vec-TextCNN_20221127203942\assets


INFO:tensorflow:Assets written to: ../model/Proposed-Word2Vec-TextCNN_20221127203942\assets


# Test

In [21]:
model = keras.models.load_model(SAVED_MODEL)

In [22]:
pred = model.predict(x_test)

2188/2188 [==============================] - 26s 12ms/step


In [23]:
y_test = [np.argmax(i) for i in y_test]
y_pred = [np.argmax(i) for i in pred]

target_names = [c[:3]+"." for c in classes]

rep = metrics.classification_report(y_test, y_pred, target_names=target_names, digits=3)
print(rep)

              precision    recall  f1-score   support

        Com.      0.834     0.537     0.653      5000
        Edu.      0.674     0.707     0.690      5000
        Art.      0.877     0.510     0.645      5000
        Ath.      0.942     0.917     0.929      5000
        Off.      0.756     0.810     0.782      5000
        Mea.      0.540     0.385     0.449      5000
        Bui.      0.825     0.386     0.526      5000
        Nat.      0.288     0.868     0.433      5000
        Vil.      0.698     0.964     0.809      5000
        Ani.      0.776     0.349     0.481      5000
        Pla.      0.790     0.535     0.638      5000
        Alb.      0.905     0.905     0.905      5000
        Fil.      0.866     0.542     0.667      5000
        Wri.      0.539     0.731     0.620      5000

    accuracy                          0.653     70000
   macro avg      0.736     0.653     0.659     70000
weighted avg      0.736     0.653     0.659     70000

